In [50]:
import pandas as pd
import altair as alt

In [23]:
nyc_df = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2018/2018-12-11/nyc_restaurants.csv")

In [51]:
nyc_df[nyc_df['grade'].notna()]

,camis,dba,boro,zipcode,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,grade,inspection_type
0,50039667,WIDDI HALL,BROOKLYN,11220.0,Middle Eastern,06/02/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,9.0,A,Pre-permit (Operational) / Re-inspection
3,50032889,SHUN YUEN CHINESE RESTAURANT,BRONX,10458.0,Chinese,03/29/2018,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,12.0,A,Cycle Inspection / Initial Inspection
6,50058588,SWEETGREEN 55TH AND PARK,MANHATTAN,10022.0,Salads,10/30/2018,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,A,Cycle Inspection / Initial Inspection
9,41591198,PEARL'S SOCIAL & BILLY CLUB,BROOKLYN,11237.0,American,03/06/2018,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,6.0,A,Cycle Inspection / Initial Inspection
10,50007219,GREENPOINT BEER & ALE,BROOKLYN,11222.0,American,10/04/2016,Violations were cited in the following area(s).,09C,Food contact surface not properly maintained.,Not Critical,7.0,A,Cycle Inspection / Initial Inspection
...,...,...,...,...,...,...,...,...,...,...,...,...,...
299993,50036379,HOUSE OF THAI,QUEENS,11101.0,Thai,06/26/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,7.0,A,Cycle Inspection / Re-inspection
299994,50018793,NEW GOLDEN DRAGON CHINESE RESTAURANT,MANHATTAN,10031.0,Chinese,07/18/2016,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,10.0,A,Cycle Inspection / Re-inspection
299996,50074209,GERMAN'S SOUP,BROOKLYN,11203.0,Caribbean,08/09/2018,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Critical,13.0,A,Cycle Inspection / Re-inspection
299998,50037983,RINCON MEXICANO,MANHATTAN,10033.0,"Latin (Cuban, Dominican, Puerto Rican, South &...",01/12/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,12.0,A,Pre-permit (Operational) / Initial Inspection
